# Qwen2.5-Omni：音视频交互中的数学聊天实践

我们体验一下如何在音频和视频流中使用Qwen2.5-Omni讨论数学内容。

# 环境准备

如果前面已经安装准备完成，跳过此章节

## 安装相关依赖
安装transformers、qwen-omni-utils、flash-attn --no-build-isolation

```bash
pip uninstall transformers
pip install git+https://github.com/huggingface/transformers@f742a644ca32e65758c3adb36225aef1731bd2a8
pip install accelerate
pip install triton
pip install qwen-omni-utils
pip install -U flash-attn --no-build-isolation
# 用于从魔塔模型库下载模型
pip install modelscope
```



## 模型下载准备

需要提前将模型下载到本地备用
使用 modelscope 中的 snapshot_download 函数下载模型（提前安装modelscope）。
第一个参数为模型名称，第二个参数 cache_dir 用于指定模型的下载路径.
```python

from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen2.5-Omni-7B', cache_dir='/root/autodl-tmp', revision='master')

```

# 加载模型&导入依赖

加载本地下载的模型，同时启用flash_attention_2加速

In [ ]:
import torch
from transformers import Qwen2_5OmniModel, Qwen2_5OmniProcessor
import librosa
import audioread
from IPython.display import Video
from IPython.display import Audio

model_path = "Qwen/Qwen2.5-Omni-7B"
model = Qwen2_5OmniModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
processor = Qwen2_5OmniProcessor.from_pretrained(model_path)

# 定义推理函数

In [ ]:
from qwen_omni_utils import process_mm_info

#  推理函数，用于处理视频输入，并生成文本和音频输出。
def inference(video_path):
    messages = [
        {"role": "system", "content": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."},
        {"role": "user", "content": [
                {"type": "video", "video": video_path},
            ]
        },
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # image_inputs, video_inputs = process_vision_info([messages])
    audios, images, videos = process_mm_info(messages, use_audio_in_video=True)
    inputs = processor(text=text, audios=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=True)
    inputs = inputs.to(model.device).to(model.dtype)

    output = model.generate(**inputs, use_audio_in_video=True, return_audio=True)

    text = processor.batch_decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    audio = output[1]
    return text, audio

/opt/conda/envs/omni/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


# 对话聊天

通过视频与 Qwen2.5-Omni 进行多模态聊天，讨论数学相关的内容

In [ ]:
video_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2.5-Omni/math.mp4"

display(Video(video_path, width=640, height=360))
display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(video_path), sr=16000)[0], rate=16000))

## Use a local HuggingFace model to inference.
response, audio  = inference(video_path)
print(response[0])

display(Audio(audio, rate=24000))

qwen-vl-utils using torchvision to read video.
Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user

assistant
Well, if you have the equation x³ + y = 12 and you know that y = 4, then you can substitute y = 4 into the equation. So, x³ + 4 = 12. Then you subtract 4 from both sides, which gives you x³ = 8. And the cube root of 8 is 2. So, x = 2. If you have any other math problems or questions, feel free to let me know.
